# Cognitive, behavioral and social data
**DATASET**: PCL5  
**Author**: Mattia Brocco

MERGE OF DATASETS FOR **R_NEO_PI**
```python
a = pd.read_excel(data_dir + "\\R_NEO_PI_Faked.xlsx")
b = pd.read_excel(data_dir + "\\R_NEO_PI_Honest.xlsx")

a.columns = [" ".join([pd.Series(a.columns).apply(lambda s: np.nan if "Unnamed"
                                                  in s else s).fillna(method = "ffill").tolist()[i],
                       a.loc[0][i]]) for i in range(len(a.columns))]
b.columns = [" ".join([pd.Series(b.columns).apply(lambda s: np.nan if "Unnamed"
                                                  in s else s).fillna(method = "ffill").tolist()[i],
                       b.loc[0][i]]) for i in range(len(b.columns))]

a = a.drop(0).reset_index(drop = True)
b = b.drop(0).reset_index(drop = True)

a["CONDITION"] = "FAKE"
b["CONDITION"] = "HONEST"

pd.concat([a, b], ignore_index = True).to_excel(data_dir + "\\R_NEO_PI.xlsx", index = False)
```

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import engine

%load_ext autoreload
%autoreload 2

data_dir = ".\\data"

pd.options.display.max_columns = 500

---
## Try to design a pipeline
***

SOURCES  
* https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/factor-analysis/
* https://en.wikipedia.org/wiki/Likelihood-ratio_test
* https://www.ibm.com/support/pages/how-can-i-get-statistics-compare-nested-models-logistic-regression-spss

In [73]:
### PIPELINE
# Organize datasets
data_collection = {}
for dataset in os.listdir(data_dir):
    print(dataset)
    a, b, c, d = engine.Classification().prepare_data(f"{data_dir}\\{dataset}", "CONDITION")
    e = engine.Classification().variable_selection(a, b, c, d)
    f = engine.Classification().benchmark_models(a, b, c, d, e)
    data_collection[dataset.split(".")[0]] = [a, b, c, d, e, f]
    print(e)
    print(f)
    print("-"*50)
    print()

BF_df_CTU.csv
5/5 [==============================] - 0s 1ms/step
{'Features': [2, 6, 4, 3], 'Validation passed': False}
Logistic Regression    0.796992
SVC                    0.819549
Random Forest          0.774436
Neural Network         0.796992
dtype: float64
--------------------------------------------------

BF_df_OU.csv
5/5 [==============================] - 0s 1ms/step
{'Features': [6, 4, 1], 'Validation passed': False}
Logistic Regression    0.789855
SVC                    0.804348
Random Forest          0.804348
Neural Network         0.775362
dtype: float64
--------------------------------------------------

BF_df_V.csv
5/5 [==============================] - 0s 1ms/step
{'Features': [6, 7], 'Validation passed': False}
Logistic Regression    0.719178
SVC                    0.726027
Random Forest          0.726027
Neural Network         0.691781
dtype: float64
--------------------------------------------------

DT_df_CC.csv
5/5 [==============================] - 0s 1ms/step
{'F